<a href="https://colab.research.google.com/github/to-afternoon/demo/blob/main/APP_FINAL_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Failure Prediction

## Importing Libraries and Data

In [1]:
!pip install streamlit

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import io
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import streamlit as st

sns.set_style("darkgrid")

In [2]:
st.title("Failure Prediction App")

2022-05-16 16:45:25.422 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [3]:
# uploading data file from local drive in google colab
from google.colab import files
uploaded = files.upload()

Saving predictive_maintenance.csv to predictive_maintenance.csv


In [4]:
data = pd.read_csv(io.BytesIO(uploaded['predictive_maintenance.csv']))

# Pre Processing for Prediction 

In [5]:
data = data.drop(["UDI",'Product ID'],axis=1)

## Encoding

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

label_encoder.fit(data['Type'])
data['Type'] = label_encoder.transform(data['Type'])

label_encoder.fit(data['Target'])
data['Target'] = label_encoder.transform(data['Target'])

## Test Tain Split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['Failure Type','Target'],axis=1),
                                                    data['Target'], test_size=0.3, random_state=42)

## Prediction

In [9]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [10]:
lr.fit(X_train, y_train)

print(lr.score(X_test, y_test))

0.9733333333333334


In [11]:
X_train

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
9069,2,297.2,308.2,1678,28.1,133
2603,2,299.3,309.2,1334,46.3,31
7738,2,300.5,312.0,1263,60.8,146
1579,1,298.3,308.3,1444,43.8,176
5058,1,303.9,312.9,1526,42.5,194
...,...,...,...,...,...,...
5734,1,302.3,311.8,1369,56.2,208
5191,1,304.0,313.2,1416,46.0,128
5390,0,302.8,312.3,1483,47.2,223
860,0,296.1,306.9,1541,32.6,33


In [21]:
Type = (st.number_input("Enter the Type of Machine"))

AirTemperature = (st.number_input("Enter the Air Temperature"))

ProcessTemperature = (st.number_input("Enter the Process Temperature"))

RotationalSpeed = (st.number_input("Enter the Rotational Speed"))

Torque = (st.number_input("Enter the Torque"))

ToolWear = (st.number_input("Enter the Tool Wear"))

In [24]:
outcome = lr.predict([[Type, AirTemperature, ProcessTemperature, RotationalSpeed, RotationalSpeed, ToolWear]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [27]:
st.header("Prediction : ")

if (outcome[0] == 0):
  st.success("No Failure")
if (outcome[0] == 1):
  st.error("Failure")

# **Type of Failure**

In [29]:
number = preprocessing.LabelEncoder()

data["Type"] = number.fit_transform(data["Type"])
data["Failure Type"] = number.fit_transform(data["Failure Type"])

In [30]:
X = data.drop(["Failure Type"], axis = 1)
y = data["Failure Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.25)

In [31]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
model = LogisticRegression(multi_class = "multinomial", solver = "saga")
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9948


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [33]:
OutcomeType = model.predict([[Type, AirTemperature, ProcessTemperature, RotationalSpeed, RotationalSpeed, ToolWear, outcome[0]]])

array([1])

In [ ]:
st.header("If any Failure occurs, chances are, the reason will be : ")

if (OutcomeType[0] == 0):
  st.info("Heat Dissipation Failure")

if (OutcomeType[0] == 1):
  st.info("No Failure")

if (OutcomeType[0] == 2):
  st.info("Overstrain Failure")

if (OutcomeType[0] == 3):
  st.info("Power Failure")

if (OutcomeType[0] == 4):
  st.info("Random Failure")

if (OutcomeType[0] == 5):
  st.info("Tool Wear Failure")